<a href="https://colab.research.google.com/github/Kzis/ML-Programming/blob/master/Homework/HW3-Classification-Logistic%20Regression/CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle


In [3]:
!ls -a

.  ..  .config	.kaggle  sample_data


In [13]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [14]:
!ls -a

.  ..  .config	.kaggle  kaggle.json  sample_data


In [12]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"driftkao","key":"1b85ac7759439a01bf8aafd05cc6a85b"}'}

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

In [17]:
!kaggle competitions download -c avazu-ctr-prediction

 98% 1.02G/1.04G [00:09<00:00, 93.7MB/s]
100% 1.04G/1.04G [00:09<00:00, 122MB/s] 
 96% 113M/118M [00:02<00:00, 56.5MB/s]
100% 118M/118M [00:02<00:00, 57.1MB/s]
 51% 17.0M/33.2M [00:00<00:00, 39.7MB/s]
100% 33.2M/33.2M [00:00<00:00, 84.0MB/s]


In [20]:
!unzip \*.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [25]:
!gunzip test.gz

gzip: test.gz: No such file or directory
